In [13]:
import os
import subprocess
import bt
import datetime as dt

tickers = subprocess.run(args=["python", os.getcwd() + "\\data_cleaning.py"],text=True,stdout=subprocess.PIPE)

results = tickers.stdout.strip('""[]\n').split(', ')

symbols = [element.strip("'\"") for element in results]

current_date = dt.date.today()
start_date = current_date - dt.timedelta(days=365)

num_previous_trading_days = sum(1 for day in range((current_date - start_date).days + 1) if (start_date + dt.timedelta(days=day)).weekday() < 5)

days = round((1.9*num_previous_trading_days)-1,0)

start = (current_date - dt.timedelta(days=days)).strftime("%Y-%m-%d")

df = bt.get(symbols,start=start)


In [43]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_torus(ticker,big,rolling_mean,rolling_std,buy_signal,sell_signal,size):
    # Define parametric equations for a torus
    def torus_parametric(u, v, R, r):
        x = (R + r * np.cos(v)) * np.cos(u)
        y = (R + r * np.cos(v)) * np.sin(u)
        z = r * np.sin(v)
        return x, y, z

    # Parameters for the torus
    R = (rolling_mean[ticker] + (2 * rolling_std[ticker])).mean()  # Major radius
    r = rolling_mean[ticker].mean()  # Minor radius

    # Generate torus coordinates
    u = np.linspace(0,big,size)
    v = np.linspace(0,big,size)
    U, V = np.meshgrid(u, v)
    X, Y, Z = torus_parametric(U, V, R, r)

    # Plot the torus
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, alpha=0.2, cmap='viridis')

    # Plot the price data
    ax.scatter(X[buy_signal[ticker]], Y[buy_signal[ticker]], Z[buy_signal[ticker]], color='g', label='Buy Signal')
    ax.scatter(X[sell_signal[ticker]], Y[sell_signal[ticker]], Z[sell_signal[ticker]], color='r', label='Sell Signal')
    ax.tick_params(labelsize=20)
    
    plt.show()

def action_tickers(ticker,buy_indicator,sell_indicator):
    tick_buy = pd.DataFrame(buy_indicator[ticker])
    tick_buy["BuySignal"] = "Buy"
    tick_buy = tick_buy.rename(columns={ticker:"BuyAction"})

    tick_sell = pd.DataFrame(sell_indicator[ticker])
    tick_sell["SellSignal"] = "Sell"
    tick_sell = tick_sell.rename(columns={ticker:"SellAction"})

    actions = pd.concat([tick_buy,tick_sell],axis=1)
                     
    actions.loc[(actions["BuyAction"] == False) & (actions["SellAction"] == False), ["BuySignal","SellSignal"]] = "Hold"

    buy = actions.loc[(actions["BuySignal"] == "Buy") & (actions["BuyAction"] == True), "BuySignal"]
    sell = actions.loc[(actions["SellSignal"] == "Sell") & (actions["SellAction"] == True), "SellSignal"]
    hold = actions.loc[(actions["BuySignal"] == "Hold") & (actions["SellSignal"] == "Hold"), "BuySignal"]

    actions = pd.concat([buy,sell,hold],axis=0).reset_index()

    actions.columns = ["Date", ticker + "Signal"]

    actions = actions.sort_values(by="Date",ascending=True).reset_index(drop=True)

    actions = actions.drop(["Date"],axis=1)

    return actions


big = 2 * np.pi

current_date = dt.date.today()
start_date = current_date - dt.timedelta(days=365)

a = df.loc[df.index >= start_date.strftime("%Y-%m-%d")]

norm_price = ((df - a.min()) / (a.max() - a.min())) * big

# Calculate the mean and standard deviation 

rolling_mean = norm_price.rolling(window=50).mean()
rolling_std = norm_price.rolling(window=50).std()
df = (df - df.loc[df.index == start_date.strftime("%Y-%m-%d")].values)

buy_signal = df < rolling_mean - (2 * rolling_std)
sell_signal = df > rolling_mean + (2 * rolling_std)

buy_signal = buy_signal.loc[buy_signal.index >= start_date.strftime("%Y-%m-%d")]
sell_signal = sell_signal.loc[sell_signal.index >= start_date.strftime("%Y-%m-%d")]

ticker_action = pd.DataFrame(buy_signal.index)

for tick in symbols:
    ticker_action[tick.lower() + "Signal"] = action_tickers(tick.lower(),buy_signal,sell_signal)

date = ticker_action.pop("Date")
ticker_action = ticker_action.set_index(date)

ticker_action.tail(n=50)

# plot_torus("pypl",big,rolling_mean.loc[rolling_mean.index >= start_date.strftime("%Y-%m-%d")],
#            rolling_std.loc[rolling_std.index >= start_date.strftime("%Y-%m-%d")],buy_signal,sell_signal,len(ticker_action))

,cdlxSignal,cvxSignal,pyplSignal,modSignal,calmSignal,indaSignal,anfSignal,ymmSignal,cortSignal,dloSignal,gntxSignal,incySignal
Date,,,,,,,,,,,,
2024-02-27,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-02-28,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-02-29,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-03-01,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-03-04,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-03-05,Hold,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-03-06,Hold,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-03-07,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy
2024-03-08,Sell,Buy,Buy,Sell,Sell,Sell,Sell,Buy,Hold,Buy,Sell,Buy


In [ ]:
def action_tickers(ticker,buy_indicator,sell_indicator):
    tick_buy = pd.DataFrame(buy_indicator[ticker])
    tick_buy["BuySignal"] = "Buy"
    tick_buy = tick_buy.rename(columns={ticker:"BuyAction"})

    tick_sell = pd.DataFrame(sell_indicator[ticker])
    tick_sell["SellSignal"] = "Sell"
    tick_sell = tick_sell.rename(columns={ticker:"SellAction"})

    actions = pd.concat([tick_buy,tick_sell],axis=1)
                     
    actions.loc[(actions["BuyAction"] == False) & (actions["SellAction"] == False), ["BuySignal","SellSignal"]] = "Hold"

    buy = actions.loc[(actions["BuySignal"] == "Buy") & (actions["BuyAction"] == True), "BuySignal"]
    sell = actions.loc[(actions["SellSignal"] == "Sell") & (actions["SellAction"] == True), "SellSignal"]
    hold = actions.loc[(actions["BuySignal"] == "Hold") & (actions["SellSignal"] == "Hold"), "BuySignal"]

    actions = pd.concat([buy,sell,hold],axis=0).reset_index()

    actions.columns = ["Date", ticker + "Signal"]

    actions = actions.sort_values(by="Date",ascending=True).reset_index(drop=True)

    actions = actions.drop(["Date"],axis=1)

    return actions

ticker_action = pd.DataFrame(df.index)

for tick in symbols:
    ticker_action[tick.lower() + "Signal"] = action_tickers(tick.lower(),buy_signal,sell_signal)

date = ticker_action.pop("Date")
ticker_action = ticker_action.set_index(date)

ticker_action